# Welcome To the Clickshot AI Documentation

Success Criteria:
- Successfully run Clickshot
- Be able to delete a square when clicked on and and have a new square generated in a random position
- Have the game go on infinitely until exited by the user
- Capture the screen and feed it into python at 30+ fps
- Have an AI play the game and be able to indefinitely beat it    

# Algorithms

**Game Loop**: 
This algorithm is the centre of the game. It recieves game events and processes them aswell as rendering images onto the screen.


 ![title](images/GameLoop.drawio.png)


**AI Loop**:
This algorithm reads images of the virtual camera and then process the images. After processing the frame it applies contour fining algorithms and then draws the conoturs onto the screen.

![title](images/VisionLoop.drawio.png)

# Anotated Development

### ClickShot.py: 
This contains the game that is played by the AI.

In [ ]:
import pygame # imports the pygame library
import random # imports the random library

This Code will import both the libraries required for the code. Pygame handles the inputs, gameloop and the graphics.

In [ ]:
pygame.init() # initializes the pygame module

screen = pygame.display.set_mode((1920, 1080)) # sets the screen to 1920x1080
screen.fill((255, 255, 255)) # fills the screen to make it white
running = True # variable to control the while loop of the game

This section of code does the basic initilization to create a window. It first sets the width and height of the window. The display object is stored in the screen variable. Then the screen is filled to be white. The running variable is set to true so that the gameloop runs indefinetly until exited.

In [ ]:
class Targets(pygame.sprite.Sprite): # inherits from the sprite class
    def __init__(self, width, height, colour, x_pos, y_pos): 
        super().__init__() # initialises base items from the sprite class
        self.image = pygame.Surface([width, height]) # sets the width and height of the rectange
        self.image.fill(colour) # colours the rectangle
        self.rect = self.image.get_rect() # stores the rectangle
        self.rect.center = (x_pos, y_pos) # sets the rectangles x and y coordinates

This piece of code defines the Target class of the game. This is the main object in our game. It stores its position and bounds. It also stores its colour. The rectangle is derived from teh surface generated and then dtored as part of the sprite.

In [ ]:
Target_group = pygame.sprite.Group() # generates a sprite group
mouse_x = 0 # variable to store the x coordinage of the mouse
mouse_y = 0 # variable to store the y coordinate of the mouse

Target = Targets(50, 50, (0, 0, 255), random.randint(1, 1920), random.randint(1, 1080)) # Target object which is 50x50, blue and has a random position on the screen
Target_group.add(Target) # Target added to the target group

This code initialises the target for the game. It first creates a target object which is 50x50, blue and has a random position on the screen. This is then added to the sprite group - "Target_group". This is the pygame methode for storing sprites to make them easier to render.

In [ ]:
while running: # The Game Loop
    for event in pygame.event.get(): # chechs for evenys
        if event.type == pygame.QUIT: # if a user quits the game
            pygame.quit()
        if event.type == pygame.MOUSEMOTION: # if the user moves the mouse
            mouse_x, mouse_y = event.pos
        if event.type == pygame.MOUSEBUTTONDOWN: # if the user clicks the mouse
            for target in Target_group: # loops through all the targets
                if target.rect.collidepoint(mouse_x, mouse_y): # Checks wether there is a collision between the mouse and the rectangle of the sprite 
                    target.kill() # target deleted from the group
                    screen.fill("white") # screen filled
                    Target = Targets(50, 50, (0, 0, 255), random.randint(1, 1920), random.randint(1, 1080)) # new target generated
                    Target_group.add(Target) # new target added
                    Target_group.draw(screen) # new target drawn onto the screen

    pygame.display.flip() # the screen is updated
    Target_group.draw(screen) # item is drawn if there are no events

This code is the main game loop. It runs indefinetly using a while true loop. Then, all the game events are fetched. These are then looped through to chech their type. If it is a "Quit event" the game is closed. If the mouse moves, the x and y positions of the mouse are stored in variables. If the mosue is clicked, there is a check to see if any of the sprites are colliding with the position of the mouse. If so, the sprite is eliminated and a new one is generated to replace it. It has a randomised position. It is then added to the sprite group and drawn on the screen. The screen us updated using pygame.display.flip(). Then the target group is drawn if there are no events registered. This loop goes on until the game is quited.

The end product of the game looks like this:

![title](images/game1.png)

As you can see below, after being clicked the new sprite is generated in a different position

![title](images/game2.png)

### Main.py: 
This runs the AI which processes images fed to it and can the provide inputs into the game

In [ ]:
import cv2
import numpy as np
import mouse

This code imports the relevent modules for the AI. OpenCV is a module used to process images and videos. Numpy is used to handle arrays, matricies and perform linear algebra operations. The mouse module allows for the AI to control the mouse and click, allowing it to play the game.

In [ ]:
cam = cv2.VideoCapture(0) # recieves inpu from teh virtual camera

This line of code creates an object that can read from a camera. For this AI, I used the OBS virtual camera to capture the screen. This is because OBS can capture the screen at 60 FPS, which is greater than trying to screenshot the screen rapidly. The OBS interface looks like this:

![title](images/obs.png)

This utilises the virtual camera plugin which allows screen recording to be viewed as a camera by Opencv.

In [ ]:
while True:
    check, img = cam.read() # frame readf of the camera
    frame = np.array(img) # coverted to an array

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # frame converted to hsv 
    mask = cv2.inRange(hsv, np.array([110, 200, 20]), np.array([130, 255, 255])) # colour mask generated

This piece of code starts the while loop. Then an image is read from the virtual camera. It is then made into a numpy array to make the processing easier. Then, the image needs to be converted to hsv format as that is the format used for colour masks. Then the image is converted into a colour mask. this detects all pixels within the range set in the function. Here is the output:

![title](images/mask.png)

As you can see, the image is converted to a black an white image where white pixels fell within the range and black pixels outsude. This creates a clear contour around the rectangle.

In [ ]:
contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    area = cv2.contourArea(cnt)
    if 3000 > area > 100:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        mouse.drag(0, 0, x, y, absolute=False, duration=0.1)
        mouse.click("left")

This next part of the code is key for the AI. "cv2.findcontours()"is a function that applies a contour finding algorithm to the image. This detects big shifts in colour between areas. This will then return a list of contour objects which contain coordinates of the contours. For each contour, the area is calculated and checked to be within a significant area. This stops small changes in colour being detected by the AI. Then, the x, y, width and height is found using the "cv2.boundingRect()". This then allows a rectangle to be drawn around the contour on the frame:

![title](images/frame.png)

Then, using the mouse module, the AI moves to the position of the contour and clicks on it.

In [ ]:
    cv2.imshow('Screen', frame)
    cv2.imshow("Identifier", mask)


    c = cv2.waitKey(1)
    if c == 33:
        break

cv2.destroyAllWindows()

This part of the code displays both the mask and the frame in real time. Then there is a check to see if space has been pressed. This will break the loop and end the AI using "cv2.destroyAllWindows".

# Evaluation

Overall, the game and AI meet all the critera set out at the start. It was especially efficent when capturing the screen. However, the AI can be imporved by adding complexity through more complex gameplay or harder to recognise shapes. In addition to this, the background could be more complex to require more precise preprocessing. In Summation, this project was a success but has quite a lot more room to be extended for more compelxity.